In [1]:
import datetime as dt
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt

# Fixed parameters

In [2]:
#number of parameter vectors
samplesize = 5000

# Variable parameters (qubits, layers and distributions)

In [3]:
#List of distributions
distributions = ['normal', 'uniform', 'exponential']

#List of number of qubits
qubits = [4, 5, 6]

#List of number of layers
layers = [6, 7, 8, 9, 10]

# Calculation of the Lorenz curves standard deviations

In [4]:
def SD_lorenz(dist, samples, circuit, nlayers):
    #Store cumulants
    CumAvg = []
    DCum = []
    
    #selects the distribution
    if dist == 'normal':

        CumAvg=0
        DCum=0
        
        for i in range(samples):
            #this part generates the vectors of angles for the circuits. This is conditioned on the number of layers
            #and qubits. The array of parameters will have two angles (one for RX and one for RY) per qubit and
            #one vector of this kind for every layer. So, for the calculation of the expressibility, we sample
            #two of these parameter vectors and then apply then into a circuit.
            
            
            angles1 = np.array([ [[np.random.normal(loc = np.pi, scale = np.pi/4) for i in range(wires)], 
                                [np.random.normal(loc = np.pi, scale = np.pi/4) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            #Here the parameter vectors are applied to generate two states considering a particular circuit
            #structure that is given as an input for the function.
            
            state1 = circuit(angles1)

            #1st Section added by GGC, SD of Lorenz curves, in samples loop, using final vector/each circuit set

            P = np.abs( state1 )**2

            #Decreasing ordering of Pcomp
            P[::-1].sort()
	
            #Cummulants calculation
            Cum=np.cumsum(P)

            #Cummulants average and squared average calculation
            CumAvg=CumAvg+Cum
            DCum=DCum+Cum**2

            #End of 1st Section added by GGC, SD of Lorenz curves
        
        #2nd Section added by GGC, SD of Lorenz curves final computation at the end of samples loop
        
        #GIC comment: here I added the np.abs because sometimes the values are really small (e-16),
        #but negative, so the square root should be zero but it can't be calculated due to the negative sign.
        SDL=np.sqrt(np.abs(DCum/samples-(CumAvg/samples)**2))
    
    
    #The same is done for the other distributions
    if dist == 'exponential':
        
        CumAvg=0
        DCum=0
        
        for i in range(samples):
        
            angles1 = np.array([ [[np.random.exponential(scale = 0.865) for i in range(wires)], 
                                [np.random.exponential(scale = 0.865) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
            #Here the parameter vectors are applied to generate two states considering a particular circuit
            #structure that is given as an input for the function.
            
            state1 = circuit(angles1)

            #1st Section added by GGC, SD of Lorenz curves, in samples loop, using final vector/each circuit set

            P = np.abs( state1 )**2

            #Decreasing ordering of Pcomp
            P[::-1].sort()
	
            #Cummulants calculation
            Cum=np.cumsum(P)

            #Cummulants average and squared average calculation
            CumAvg=CumAvg+Cum
            DCum=DCum+Cum**2

            #End of 1st Section added by GGC, SD of Lorenz curves
        
        #2nd Section added by GGC, SD of Lorenz curves final computation at the end of samples loop
        SDL=np.sqrt(np.abs(DCum/samples-(CumAvg/samples)**2))
        
    if dist == 'uniform':
        
        CumAvg=0
        DCum=0
    
        for i in range(samples):
        
            angles1 = np.array([ [[np.random.uniform(low=0, high=2*np.pi) for i in range(wires)], 
                                [np.random.uniform(low=0, high=2*np.pi) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
            #Here the parameter vectors are applied to generate two states considering a particular circuit
            #structure that is given as an input for the function.
            
            state1 = circuit(angles1)

            #1st Section added by GGC, SD of Lorenz curves, in samples loop, using final vector/each circuit set

            P = np.abs( state1 )**2

            #Decreasing ordering of Pcomp
            P[::-1].sort()
	
            #Cummulants calculation
            Cum=np.cumsum(P)

            #Cummulants average and squared average calculation
            CumAvg=CumAvg+Cum
            DCum=DCum+Cum**2

            #End of 1st Section added by GGC, SD of Lorenz curves
        
        #2nd Section added by GGC, SD of Lorenz curves final computation at the end of samples loop
        SDL=np.sqrt(np.abs(DCum/samples-(CumAvg/samples)**2))
        
    return SDL

# Code that executes the different conditions

In [5]:
#In this part of the code, the circuits are executed with different distributions, number of circuit concatenations
#(layers) and different number of qubits. For each set of parameters, the data is stored in a .txt file



now0 = dt.datetime.now() #monitorar tempo de início
now0 = now0.strftime("%Y-%m-%d %H:%M:%S")
print("Tempo inicial: ")
print(now0)
print()

for dist in distributions:
    
    for wires in qubits:
        #This command initializes the quantum circuit of qubits in pennylane, with number of qubits given by wires
        #and considering analytical computations (shots=None), if we want to compute mean values of observables.
        #shots different from None would lead to a stochastic calculation of mean values.
        
        dev = qml.device("default.qubit", wires=wires, shots=None)
        
        
        for nlayers in layers:
        
            #No connections
            
            #To define a circuit with different number of layers in pennylane, we need to define which is
            #the structure of each layer (layer_noconnec function below) and then define how it is concatenated.
            #As we are just stacking the layers, changing only the values of the parameters, we just have to tell
            #that we want it to make a layered circuit (noconnec) with the structure of layer_noconnec.
            #The return qml.state() means that the output of the circuit is the complete quantum state generated
            #in the form of a vector.
            
            def layer_noconnec(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])
        
            def noconnec(rotations):
                qml.layer(layer_noconnec, nlayers, rotations)
                return qml.state()

            
            #In this part we are combining the defined circuit with the defined device, a.k.a. the qubits quantum
            #circuit.
            noconnec_circuit = qml.QNode(noconnec, dev)
        
        
            #The same is done with the other circuits, but now we add the CNOTs step. Each one of them will have a
            #different command to generate the couplings as we want.
            
            
            #Linear
            def layer_linear(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])
                qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")

            def linear(rotations):
                qml.layer(layer_linear, nlayers, rotations)
                return qml.state()

            linear_circuit = qml.QNode(linear, dev)
        
        
        
            #Ring
            def layer_ring(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])
                qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")
                qml.CNOT(wires=[wires-1,0])
    
            def ring(rotations):
                qml.layer(layer_ring, nlayers, rotations)
                return qml.state()

            ring_circuit = qml.QNode(ring, dev)
        
        
        
            #Star
            def layer_star(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])
                for i in range(wires-1):
                    qml.CNOT(wires = [0, i+1])

            def star(rotations):
                qml.layer(layer_star, nlayers, rotations)
                return qml.state()

            star_circuit = qml.QNode(star, dev)

        #4th modif. added by GGC, SD of Lorenz curves recovered from fidel_histogram to use the same output file

            SDL_noconnec = SD_lorenz(dist, samplesize, noconnec_circuit, nlayers)
            SDL_linear = SD_lorenz(dist, samplesize, linear_circuit, nlayers)
            SDL_ring = SD_lorenz(dist, samplesize, ring_circuit, nlayers)
            SDL_star = SD_lorenz(dist, samplesize, star_circuit, nlayers)


            f = open("expressibility_ansatz1_dist{0}_qubits{1}_layers{2}_samples{3}.txt"
                     .format(dist,wires,nlayers,samplesize), "w")

        #5th modif. added by GGC, SD of Lorenz curves output
            
            print('Circuit,StDevLorenz', file = f)
            print('No connections,', SDL_noconnec, file = f)
            print('Linear,', SDL_linear, file = f)
            print('Ring,', SDL_ring, file = f)
            print('Star,', SDL_star, file = f)

            
            print('Execution: distribution=',dist,'; qubits=',wires,'; layers=', nlayers)
            f.close()
            now1 = dt.datetime.now() #monitorar tempo de início
            now1 = now1.strftime("%Y-%m-%d %H:%M:%S")
            print("Tempo: ",now1)
            print()
     
    
now1 = dt.datetime.now() #monitorar tempo de início
now1 = now1.strftime("%Y-%m-%d %H:%M:%S")

f.close()

print()
print("Tempo inicial: ")
print(now0)
print("Tempo final:")
print(now1)

Tempo inicial: 
2024-01-24 09:34:07

[2.23808114e-02 2.50276785e-02 2.03835625e-02 1.58329310e-02
 1.10999777e-02 7.64293294e-03 5.23228280e-03 3.52565827e-03
 2.26750724e-03 1.35682490e-03 7.64335838e-04 3.89588476e-04
 1.79564758e-04 6.17283175e-05 1.24336089e-05 0.00000000e+00]
[2.23808114e-02 2.50276785e-02 2.03835625e-02 1.58329310e-02
 1.10999777e-02 7.64293294e-03 5.23228280e-03 3.52565827e-03
 2.26750724e-03 1.35682490e-03 7.64335838e-04 3.89588476e-04
 1.79564758e-04 6.17283175e-05 1.24336089e-05 0.00000000e+00]
[1.76872363e-02 2.06589102e-02 1.56585506e-02 1.20224650e-02
 7.92925255e-03 5.38141124e-03 3.65854160e-03 2.40426661e-03
 1.46969944e-03 8.26059090e-04 4.23149353e-04 2.02352356e-04
 8.84301551e-05 2.97711483e-05 5.35471392e-06 0.00000000e+00]
[1.76872363e-02 2.06589102e-02 1.56585506e-02 1.20224650e-02
 7.92925255e-03 5.38141124e-03 3.65854160e-03 2.40426661e-03
 1.46969944e-03 8.26059090e-04 4.23149353e-04 2.02352356e-04
 8.84301551e-05 2.97711483e-05 5.35471392e-06

/home/ifsc-dosp21/.local/lib/python3.10/site-packages/pennylane/numpy/tensor.py:155: RuntimeWarning: invalid value encountered in sqrt
  res = super().__array_ufunc__(ufunc, method, *args, **kwargs)


[3.51764499e-03 4.93690404e-03 5.74887201e-03 5.58858554e-03
 5.11873208e-03 4.38667062e-03 3.69475483e-03 2.87798262e-03
 2.12854107e-03 1.43680923e-03 8.93951259e-04 4.90191993e-04
 2.19361166e-04 8.10992007e-05 1.77589225e-05 0.00000000e+00]
[3.51764499e-03 4.93690404e-03 5.74887201e-03 5.58858554e-03
 5.11873208e-03 4.38667062e-03 3.69475483e-03 2.87798262e-03
 2.12854107e-03 1.43680923e-03 8.93951259e-04 4.90191993e-04
 2.19361166e-04 8.10992007e-05 1.77589225e-05 0.00000000e+00]
[ 3.26376829e-03  5.00856106e-03  5.38065194e-03  5.01241758e-03
  4.35048907e-03  3.59252916e-03  2.81888138e-03  2.21060060e-03
  1.69233271e-03  1.17437516e-03  7.26453176e-04  4.22288040e-04
  2.03584996e-04  7.44230475e-05  1.37452350e-05 -3.33066907e-16]
[3.26376829e-03 5.00856106e-03 5.38065194e-03 5.01241758e-03
 4.35048907e-03 3.59252916e-03 2.81888138e-03 2.21060060e-03
 1.69233271e-03 1.17437516e-03 7.26453176e-04 4.22288040e-04
 2.03584996e-04 7.44230475e-05 1.37452350e-05 3.33066907e-16]
[8.5

[4.48555622e-03 8.20385327e-03 9.83345097e-03 1.06861123e-02
 1.08475207e-02 1.06108873e-02 1.01822183e-02 9.56707358e-03
 8.61834929e-03 7.63772963e-03 6.65299131e-03 5.75568633e-03
 4.88150072e-03 4.06737989e-03 3.35323818e-03 2.73477499e-03
 2.21590674e-03 1.76145895e-03 1.37879521e-03 1.05642774e-03
 7.88446085e-04 5.75414223e-04 4.11118846e-04 2.85131162e-04
 1.85986993e-04 1.12256240e-04 6.37687006e-05 3.30805082e-05
 1.49842663e-05 5.44872238e-06 1.02186184e-06 0.00000000e+00]
[4.48555622e-03 8.20385327e-03 9.83345097e-03 1.06861123e-02
 1.08475207e-02 1.06108873e-02 1.01822183e-02 9.56707358e-03
 8.61834929e-03 7.63772963e-03 6.65299131e-03 5.75568633e-03
 4.88150072e-03 4.06737989e-03 3.35323818e-03 2.73477499e-03
 2.21590674e-03 1.76145895e-03 1.37879521e-03 1.05642774e-03
 7.88446085e-04 5.75414223e-04 4.11118846e-04 2.85131162e-04
 1.85986993e-04 1.12256240e-04 6.37687006e-05 3.30805082e-05
 1.49842663e-05 5.44872238e-06 1.02186184e-06 0.00000000e+00]
[2.18079483e-03 4.0530

[8.91018272e-04 1.85462074e-03 2.41031851e-03 2.78911195e-03
 2.82540225e-03 2.74658003e-03 2.71874339e-03 2.66630811e-03
 2.56093704e-03 2.36060510e-03 2.14342940e-03 1.93473332e-03
 1.75184253e-03 1.55008953e-03 1.33317181e-03 1.10938861e-03
 9.12737520e-04 7.39187276e-04 5.83985701e-04 4.52372255e-04
 3.46139450e-04 2.57183192e-04 1.87143893e-04 1.33519642e-04
 9.55007779e-05 6.37015394e-05 3.84219554e-05 2.06114562e-05
 1.03433508e-05 3.74461498e-06 7.51207754e-07 2.22044605e-16]
[8.91018272e-04 1.85462074e-03 2.41031851e-03 2.78911195e-03
 2.82540225e-03 2.74658003e-03 2.71874339e-03 2.66630811e-03
 2.56093704e-03 2.36060510e-03 2.14342940e-03 1.93473332e-03
 1.75184253e-03 1.55008953e-03 1.33317181e-03 1.10938861e-03
 9.12737520e-04 7.39187276e-04 5.83985701e-04 4.52372255e-04
 3.46139450e-04 2.57183192e-04 1.87143893e-04 1.33519642e-04
 9.55007779e-05 6.37015394e-05 3.84219554e-05 2.06114562e-05
 1.03433508e-05 3.74461498e-06 7.51207754e-07 2.22044605e-16]
[4.90484797e-03 8.1101

[1.52778354e-02 2.28924065e-02 2.49156805e-02 2.49119064e-02
 2.36927677e-02 2.19465358e-02 1.98109744e-02 1.79226554e-02
 1.62021334e-02 1.46543482e-02 1.32665437e-02 1.19594021e-02
 1.07827513e-02 9.70951917e-03 8.74724769e-03 7.89216802e-03
 7.05474977e-03 6.29072817e-03 5.61385361e-03 4.99999574e-03
 4.43069987e-03 3.91987732e-03 3.46496489e-03 3.05914626e-03
 2.70661224e-03 2.39606421e-03 2.11654152e-03 1.86548570e-03
 1.64262480e-03 1.44651474e-03 1.26844048e-03 1.10910000e-03
 9.71501409e-04 8.47881441e-04 7.36231452e-04 6.38836332e-04
 5.53120324e-04 4.76376499e-04 4.08301503e-04 3.49018582e-04
 2.99113357e-04 2.55041876e-04 2.16149432e-04 1.82140797e-04
 1.51981452e-04 1.25506909e-04 1.03445605e-04 8.42589997e-05
 6.78016097e-05 5.37990095e-05 4.23582725e-05 3.27026372e-05
 2.47993566e-05 1.81860293e-05 1.31912997e-05 9.26029948e-06
 6.19872068e-06 3.93996692e-06 2.40768118e-06 1.38926673e-06
 6.79595941e-07 2.41272211e-07 4.45237702e-08 0.00000000e+00]
[1.52778354e-02 2.28924

[5.35453966e-03 1.11387112e-02 1.39773882e-02 1.56363349e-02
 1.59139149e-02 1.60288698e-02 1.59590391e-02 1.56431145e-02
 1.49257635e-02 1.41278445e-02 1.32847050e-02 1.25113389e-02
 1.17333756e-02 1.10275048e-02 1.03514267e-02 9.68305868e-03
 9.02250461e-03 8.38176975e-03 7.75619268e-03 7.17448700e-03
 6.61959527e-03 6.09852479e-03 5.60210430e-03 5.13719848e-03
 4.70103198e-03 4.28876327e-03 3.90373878e-03 3.54069898e-03
 3.19955223e-03 2.87867825e-03 2.58479882e-03 2.30719468e-03
 2.05310467e-03 1.82182888e-03 1.60849341e-03 1.41451395e-03
 1.23805614e-03 1.07968947e-03 9.37434724e-04 8.09259350e-04
 6.94813504e-04 5.93151746e-04 5.02096545e-04 4.22174416e-04
 3.51712642e-04 2.90748296e-04 2.38054744e-04 1.91973809e-04
 1.53231791e-04 1.20407416e-04 9.30889909e-05 7.01637953e-05
 5.19556218e-05 3.76037862e-05 2.68828137e-05 1.88057834e-05
 1.27045675e-05 8.08359943e-06 4.93665939e-06 2.81340035e-06
 1.40433369e-06 5.42538139e-07 1.11835395e-07 0.00000000e+00]
[5.35453966e-03 1.11387

[9.43718164e-03 1.90009370e-02 2.35831810e-02 2.57273800e-02
 2.62410029e-02 2.61317068e-02 2.50328539e-02 2.37498316e-02
 2.22933098e-02 2.07668751e-02 1.93144478e-02 1.78973312e-02
 1.65601082e-02 1.52562909e-02 1.40213522e-02 1.28555682e-02
 1.17925288e-02 1.07785819e-02 9.82754703e-03 8.93949915e-03
 8.12642928e-03 7.36662842e-03 6.67986295e-03 6.03553614e-03
 5.45196498e-03 4.91353881e-03 4.41968986e-03 3.96163691e-03
 3.55075999e-03 3.17267223e-03 2.82811696e-03 2.51049080e-03
 2.23011676e-03 1.97222521e-03 1.73960327e-03 1.52816378e-03
 1.34096251e-03 1.17062574e-03 1.01791025e-03 8.79565126e-04
 7.60205974e-04 6.51965168e-04 5.56628454e-04 4.71317203e-04
 3.96767886e-04 3.31083251e-04 2.74593734e-04 2.25097834e-04
 1.84139318e-04 1.48436448e-04 1.18648637e-04 9.31423432e-05
 7.25546038e-05 5.51401096e-05 4.11334644e-05 2.96795249e-05
 2.09848915e-05 1.40685558e-05 8.85533838e-06 4.97715558e-06
 2.52825130e-06 9.74106185e-07 2.24105781e-07 0.00000000e+00]
[9.43718164e-03 1.90009

[ 6.13138829e-04  1.30380453e-03  1.96753593e-03  2.46015456e-03
  2.73582133e-03  2.87286925e-03  2.86212122e-03  2.86573606e-03
  2.86354830e-03  2.82323253e-03  2.75129559e-03  2.66430290e-03
  2.58252372e-03  2.48723903e-03  2.39014591e-03  2.28838105e-03
  2.17961399e-03  2.06095572e-03  1.94371910e-03  1.82168481e-03
  1.70881116e-03  1.60288550e-03  1.50365663e-03  1.40600363e-03
  1.31578259e-03  1.23469426e-03  1.15353038e-03  1.07444465e-03
  9.98183682e-04  9.19238017e-04  8.42168076e-04  7.72151482e-04
  7.02628561e-04  6.39571342e-04  5.79324274e-04  5.23609396e-04
  4.73022373e-04  4.24126240e-04  3.77511445e-04  3.33383633e-04
  2.92569500e-04  2.53869003e-04  2.19272198e-04  1.87999072e-04
  1.59917244e-04  1.34835980e-04  1.11400039e-04  9.08441119e-05
  7.32234252e-05  5.84052950e-05  4.56242602e-05  3.48603226e-05
  2.60626974e-05  1.88147931e-05  1.32289609e-05  9.09834240e-06
  6.03819789e-06  3.81781411e-06  2.26548390e-06  1.20934177e-06
  5.57523378e-07  2.06542

KeyboardInterrupt: 